<h1 align=center><font size = 5>Regression</font></h1>

In [40]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense

Let's download the data and read it into a <em>pandas</em> dataframe.

In [5]:
concrete_data = pd.read_csv('concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

Let's check the dataset for any missing values.

In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

In [8]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [10]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation. The goal of normalization is to change the values of numeric columns in the dataset to use a common scale, without distorting differences in the ranges of values or losing information. For example, assume your input dataset contains one column with values ranging from 0 to 1, and another column with values ranging from 10,000 to 100,000. The great difference in the scale of the numbers could cause problems when you attempt to combine the values as features during modeling.

In [11]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [20]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

<a id="item1"></a>

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [23]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,))) # 50 neurons in input layer
    model.add(Dense(50, activation='relu')) # 50 nuerons in hidden layer
    model.add(Dense(1)) # 1 neuron inputput layer
    model.compile(optimizer='adam', loss='mean_squared_error') # Compiling
    
    return model

<a id="item4"></a>

## Train and Test the Network

In [41]:
model = regression_model()

# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Train on 721 samples, validate on 309 samples
Epoch 1/100
 - 1s - loss: 1670.8161 - val_loss: 1160.2741
Epoch 2/100
 - 0s - loss: 1563.1470 - val_loss: 1060.7175
Epoch 3/100
 - 0s - loss: 1392.9826 - val_loss: 900.9378
Epoch 4/100
 - 0s - loss: 1116.9659 - val_loss: 683.9224
Epoch 5/100
 - 0s - loss: 762.0126 - val_loss: 445.7414
Epoch 6/100
 - 0s - loss: 432.5769 - val_loss: 283.5193
Epoch 7/100
 - 0s - loss: 282.1253 - val_loss: 212.6263
Epoch 8/100
 - 0s - loss: 238.4745 - val_loss: 202.3536
Epoch 9/100
 - 0s - loss: 221.3700 - val_loss: 196.4466
Epoch 10/100
 - 0s - loss: 209.1086 - val_loss: 192.9892
Epoch 11/100
 - 0s - loss: 199.8242 - val_loss: 190.9376
Epoch 12/100
 - 0s - loss: 191.9775 - val_loss: 187.3751
Epoch 13/100
 - 0s - loss: 186.0325 - val_loss: 184.8440
Epoch 14/100
 - 0s - loss: 180.1268 - val_loss: 182.5535
Epoch 15/100
 - 0s - loss: 175.6539 - val_loss: 182.9055
Epoch 16/100
 - 0s - loss: 171.2508 - val_loss: 178.8920
Epoch 17/100
 - 0s - loss: 167.5823 - val_los